# Overview

Целью конкурса по классификации подтипов рака яичников UBC и выявлению выбросов (UBC-OCEAN) является классификация подтипов рака яичников. 
Вы создадите модель, обученную на основе самого обширного в мире набора данных о раке яичников, состоящего из гистопатологических изображений, полученных из более чем 20 медицинских центров.

Ваша работа поможет повысить применимость и доступность точных диагнозов рака яичников.

# Description
Карцинома яичников является наиболее смертельным видом рака женской репродуктивной системы. Существует пять распространенных подтипов рака яичников: серозная карцинома высокой степени, светлоклеточная карцинома яичников, эндометриоидная, серозная низкой степени и муцинозная карцинома. Кроме того, существует несколько редких подтипов ("выбросов"). Все они характеризуются различными клеточными морфологиями, этиологиями, молекулярными и генетическими профилями и клиническими признаками. Подходы к лечению, специфичные для подтипа, приобретают все большее значение, хотя сначала требуется идентификация подтипа - процесс, который можно было бы улучшить с помощью науки о данных.

В настоящее время диагностика рака яичников основывается на оценке подтипов патологами. Однако это сопряжено с рядом проблем, включая разногласия между наблюдателями и воспроизводимость диагностики. Кроме того, недостаточно обслуживаемые сообщества часто не имеют доступа к специалистам-патологоанатомам, и даже хорошо развитые сообщества сталкиваются с нехваткой патологоанатомов, обладающих опытом в области гинекологических злокачественных новообразований.

Модели глубокого обучения продемонстрировали замечательное мастерство в анализе гистопатологических изображений. Однако проблемы все еще существуют, такие как потребность в значительном объеме обучающих данных, в идеале из одного источника. Технические, этические и финансовые ограничения, а также соображения конфиденциальности усложняют обучение. В этом конкурсе у вас будет доступ к самому обширному и разнообразному набору данных о раке яичников, состоящему из гистопатологических изображений из более чем 20 центров на четырех континентах.


6 октября 2023 - Дата начала.

27 декабря 2023 - Крайний срок подачи заявок. Вы должны принять правила конкурса до этой даты, чтобы принять участие.

27 декабря 2023 г. - Крайний срок объединения команд. Это последний день, когда участники могут присоединиться к командам или объединить их.

3 января 2024 г. - Окончательный срок подачи заявок.

Все крайние сроки истекают в 11:59 по Гринвичу соответствующего дня, если не указано иное. Организаторы конкурса оставляют за собой право обновлять график проведения конкурса, если сочтут это необходимым.

- V6 ДОбавляем

* from albumentations import HorizontalFlip, RandomBrightnessContrast # new
* HorizontalFlip(p=0.5),  # Случайное горизонтальное отражение
* RandomBrightnessContrast(p=0.5),  # Случайное изменение яркости/контрастности

-V11
 "valid_batch_size": 12,(было 4)


In [ ]:
# Основные модули и утилиты
import os         # Работа с функциональностью операционной системы
import gc         # Управление сборщиком мусора Python
import math       # Математические функции
import copy       # Создание копий объектов
import cv2
import time       # Работа со временем
import random     # Генерация случайных чисел
import glob       # Поиск файлов по шаблону
from PIL import Image  # Работа с изображениями
from matplotlib import pyplot as plt  # Визуализация данных

# Обработка данных
import numpy as np   # Работа с массивами и матрицами
import pandas as pd  # Обработка и анализ табличных данных

# PyTorch и связанные с ним библиотеки
import torch                    # Основная библиотека PyTorch
import torch.nn as nn           # Модули для построения нейронных сетей
import torch.optim as optim     # Оптимизаторы для обучения моделей
import torch.nn.functional as F # Функции активации и другие утилиты
from torch.optim import lr_scheduler  # Планировщик скорости обучения
from torch.utils.data import Dataset, DataLoader  # Утилиты для работы с данными
from torch.cuda import amp      # Ускорение вычислений с помощью GPU
import torchvision              # Обработка изображений и предобученные модели

# Вспомогательные утилиты
import joblib           # Сериализация объектов
from tqdm import tqdm   # Прогресс-бар для циклов
from collections import defaultdict  # Словари с автоматическим созданием значений по умолчанию

# Scikit-Learn для обработки данных
from sklearn.preprocessing import LabelEncoder      # Кодирование меток классов
from sklearn.model_selection import StratifiedKFold  # Стратифицированное разбиение данных на фолды

# Работа с моделями изображений
import timm  # Библиотека предобученных моделей для обработки изображений

# Albumentations для аугментации изображений
import albumentations as A                  # Библиотека для аугментации изображений
from albumentations.pytorch import ToTensorV2  # Преобразование изображений в тензоры PyTorch
from albumentations import HorizontalFlip, RandomBrightnessContrast # new

# Для цветного текста в терминале
from colorama import Fore, Back, Style  # Модуль для стилизации текста в терминале
b_ = Fore.BLUE  # Синий цвет текста
sr_ = Style.RESET_ALL  # Сброс стиля текста

# Управление предупреждениями и ошибками
import warnings
warnings.filterwarnings("ignore")  # Игнорирование предупреждений
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # Для более понятных сообщений об ошибках при работе с CUDA


In [ ]:
CONFIG = {
    "seed": 42,
    "img_size": 2048,
    "model_name": "tf_efficientnet_b0_ns",
    "num_classes": 5,
    "valid_batch_size": 12,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
}

In [ ]:
def set_seed(seed=42):
    '''Устанавливает начальное значение  всего ноутбука при каждом запуске.
    Это важно для ВОСПРОИЗВОДИМОСТИ результатов.'''
    
    np.random.seed(seed)  # Установка начального значения для генератора случайных чисел NumPy

    torch.manual_seed(seed)  # Установка начального значения для генератора случайных чисел PyTorch

    torch.cuda.manual_seed(seed)  # Установка начального значения для CUDA (GPU)

    # При использовании CuDNN backend необходимо установить дополнительные параметры
    torch.backends.cudnn.deterministic = True  # Гарантирует, что операции будут детерминированными
    torch.backends.cudnn.benchmark = False    # Отключает оптимизацию производительности для детерминированных результатов

    # Установка фиксированного значения для PYTHONHASHSEED
    os.environ['PYTHONHASHSEED'] = str(seed)  # Это влияет на хеширование строк и объектов в Python

# Вызов функции с начальным значением из конфигурации
set_seed(CONFIG['seed'])


##  Данные

In [ ]:
# данные 
ROOT_DIR = '/kaggle/input/UBC-OCEAN'
TEST_DIR = '/kaggle/input/UBC-OCEAN/test_thumbnails'
# Модельи веса
LABEL_ENCODER_BIN = "/kaggle/input/ubc-efficienetnetb0-fold1of10-2048pix-thumbnails/label_encoder.pkl"
BEST_WEIGHT = "/kaggle/input/ubc-efficienetnetb0-fold1of10-2048pix-thumbnails/Recall0.9178_Acc0.9437_Loss0.1685_epoch9.bin"

In [ ]:
def get_test_file_path(image_id):
    return f"{TEST_DIR}/{image_id}_thumbnail.png"

In [ ]:
df = pd.read_csv(f"{ROOT_DIR}/test.csv")
df['file_path'] = df['image_id'].apply(get_test_file_path)
df['label'] = 0 # dummy
df

In [ ]:
df_sub = pd.read_csv(f"{ROOT_DIR}/sample_submission.csv")
df_sub

In [ ]:
encoder = joblib.load( LABEL_ENCODER_BIN )

In [ ]:
def get_cropped_images(file_path, image_id, th_area = 1000):
    image = Image.open(file_path)# Открытие изображения
     # Вычисление аспектного соотношения изображения
    as_ratio = image.size[0] / image.size[1]
    # Инициализация списков для координат кадрирования
    sxs, exs, sys, eys = [],[],[],[]
    # Проверка, если аспектное соотношение больше или равно 1.5
    if as_ratio >= 1.5:
        # Создание маски изображения
        mask = np.max( np.array(image) > 0, axis=-1 ).astype(np.uint8)
        retval, labels = cv2.connectedComponents(mask)
        # Проверка условия для кадрирования
        if retval >= as_ratio:
            x, y = np.meshgrid( np.arange(image.size[0]), np.arange(image.size[1]) )
            # Проход по каждому соединённому компоненту
            for label in range(1, retval):
                area = np.sum(labels == label)
                if area < th_area:
                    continue
                xs, ys= x[ labels == label ], y[ labels == label ]
                sx, ex = np.min(xs), np.max(xs)
                cx = (sx + ex) // 2
                crop_size = image.size[1]
                # Вычисление координат для кадрирования
                sx = max(0, cx-crop_size//2)
                ex = min(sx + crop_size - 1, image.size[0]-1)
                sx = ex - crop_size + 1
                sy, ey = 0, image.size[1]-1
                # Добавление координат в списки
                sxs.append(sx)
                exs.append(ex)
                sys.append(sy)
                eys.append(ey)
        else:
            crop_size = image.size[1]
            for i in range(int(as_ratio)):
                sxs.append( i * crop_size )
                exs.append( (i+1) * crop_size - 1 )
                sys.append( 0 )
                eys.append( crop_size - 1 )
    else:
        # Если изображение не требует кадрирования (вся картинка)
        sxs, exs, sys, eys = [0,],[image.size[0]-1],[0,],[image.size[1]-1]
# Создание DataFrame для хранения информации о кадрировании
    df_crop = pd.DataFrame()
    df_crop["image_id"] = [image_id] * len(sxs)
    df_crop["file_path"] = [file_path] * len(sxs)
    df_crop["sx"] = sxs
    df_crop["ex"] = exs
    df_crop["sy"] = sys
    df_crop["ey"] = eys
       # Возврат 
    return df_crop

In [ ]:
dfs = []
for (file_path, image_id) in zip(df["file_path"], df["image_id"]):
    dfs.append( get_cropped_images(file_path, image_id) )

df_crop = pd.concat(dfs)
df_crop["label"] = 0 # dummy
df_crop

In [ ]:
df_crop = df_crop.drop_duplicates(subset=["image_id", "sx", "ex", "sy", "ey"]).reset_index(drop=True)
df_crop

In [ ]:
class UBCDataset(Dataset):
    def __init__(self, df, transforms=None):
        # Инициализация
        self.df = df  # DataFrame с данными
        self.file_names = df['file_path'].values  # Пути к файлам изображений
        self.labels = df['label'].values  # Метки классов
        self.transforms = transforms  # Преобразования (аугментации) изображений
        # Координаты для кадрирования изображений
        self.sxs = df["sx"].values
        self.exs = df["ex"].values
        self.sys = df["sy"].values
        self.eys = df["ey"].values
        
    def __len__(self):
        # Возвращает общее количество элементов в наборе данных
        return len(self.df)
    
    def __getitem__(self, index):
        # Получение элемента по индексу
        img_path = self.file_names[index]  # Путь к изображению
        # Координаты для кадрирования
        sx = self.sxs[index]
        ex = self.exs[index]
        sy = self.sys[index]
        ey = self.eys[index]
        img = cv2.imread(img_path)  # Чтение изображения
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Преобразование в RGB
        label = self.labels[index]  # Метка класса

        # Кадрирование изображения
        img = img[sy:ey, sx:ex, :]
        
        # Применение аугментаций, если они указаны
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        # Возвращение словаря с обработанным изображением и меткой
        return {
            'image': img,
            'label': torch.tensor(label, dtype=torch.long)
        }


# Augmentations

A.RandomBrightnessContrast()

In [ ]:


data_transforms = {
    # Изменение размера изображения
    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
#         HorizontalFlip(p=0.5),  # Случайное горизонтальное отражение
#        RandomBrightnessContrast(p=0.8),  # Случайное изменение яркости/контрастности
        # Нормализация изображения
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.230, 0.223, 0.225], max_pixel_value=255.0, p=1),
        ToTensorV2()
    ], p=1.)
}

In [ ]:
# data_transforms = {
#     "valid": A.Compose([
#         # Изменение размера изображения
#         A.Resize(CONFIG['img_size'], CONFIG['img_size']),      
#         # Нормализация изображения
#         A.Normalize(
#             mean=[0.485, 0.456, 0.406],  # Средние значения для нормализации (стандартные для ImageNet)
#             std=[0.230, 0.223, 0.225],   # Стандартные отклонения (стандартные для ImageNet)
#             max_pixel_value=255.0,       # Максимальное значение пикселя
#             p=1.0                        # Вероятность применения этого преобразования
#         ),
#         ToTensorV2()  # Преобразование изображения в тензор PyTorch
#     ], p=1.)
# }


# Pooling

- Метод __init__ инициализирует модуль с заданными параметрами p и eps. Параметр p определяет степень, в которую возводятся значения пикселей перед усреднением, и он делается обучаемым параметром через nn.Parameter.

- Метод forward определяет прямой проход (forward pass) модуля. Он вызывает метод gem с текущими значениями x, p и eps.

- Метод gem реализует обобщённое среднее пулинга. Это делается путём возведения в степень p значений пикселей изображения, применения операции усреднения, и затем возведения результата в степень 1/p. Для стабильности вычислений используется значение eps.

- Метод __repr__ предоставляет строковое представление объекта, что полезно для отладки и логирования.

In [ ]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        # Инициализация параметра p как обучаемого параметра
        self.p = nn.Parameter(torch.ones(1) * p)
        # Маленькое положительное число для избегания деления на ноль
        self.eps = eps

    def forward(self, x):
        # Определение операции прямого прохода
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):#  v4 поменял p=3.5эффект не понятен
        # Реализация обобщённого среднего пулинга
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        # Представление класса в виде строки
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'


# Model_v1

In [ ]:
# class UBCModel(nn.Module):
#     def __init__(self, model_name, num_classes, pretrained=False, checkpoint_path=None):
#         super(UBCModel, self).__init__()
#         # Создание модели с использованием библиотеки timm
#         self.model = timm.create_model(model_name, pretrained=pretrained)

#         # Получение количества признаков на выходе классификатора
#         in_features = self.model.classifier.in_features
#         # Замена классификатора и глобального пула на Identity (удаление)
#         self.model.classifier = nn.Identity()
#         self.model.global_pool = nn.Identity()
#         # Добавление GeM пулинга
#         self.pooling = GeM()
#         # Линейный слой для классификации
#         self.linear = nn.Linear(in_features, num_classes)
#         # Слой Softmax для выхода вероятностей
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, images):
#         # Прямой проход через модель
#         features = self.model(images)
#         # Применение GeM пулинга и сглаживание
#         pooled_features = self.pooling(features).flatten(1)
#         # Применение линейного слоя
#         output = self.linear(pooled_features)
#         return output

# # Создание экземпляра модели с параметрами из конфигурации
# model = UBCModel(CONFIG['model_name'], CONFIG['num_classes'])
# # Загрузка предобученных весов, если они есть
# model.load_state_dict(torch.load(BEST_WEIGHT))
# # Перемещение модели на устройство для вычислений (например, на GPU)
# model.to(CONFIG['device']);

In [1]:
class UBCModel(nn.Module):
    def __init__(self, model_name, num_classes, pretrained=False, checkpoint_path=None):
        super(UBCModel, self).__init__()
        # Создание модели с использованием библиотеки timm
        self.model = timm.create_model(model_name, pretrained=pretrained)

        # Получение количества признаков на выходе классификатора
        in_features = self.model.classifier.in_features
        # Замена классификатора и глобального пула на Identity (удаление)
        self.model.classifier = nn.Identity()
        self.model.global_pool = nn.Identity()
        # Добавление GeM пулинга
        self.pooling = GeM()
        # Линейный слой для классификации
        self.linear = nn.Linear(in_features, num_classes)
        
        #***********************************************
        self.batch_norm = nn.BatchNorm1d(num_classes)  # Добавление Batch Normalization
        self.dropout = nn.Dropout(0.55)  # Добавление Dropout
        #***********************************************
        # Слой Softmax для выхода вероятностей
        self.softmax = nn.Softmax(dim=1)

    def forward(self, images):
        # Прямой проход через модель
        features = self.model(images)
        # Применение GeM пулинга и сглаживание
        pooled_features = self.pooling(features).flatten(1)
        # Применение линейного слоя
        output = self.linear(pooled_features)
        
        #***********************************************
        output = self.batch_norm(output)  # Применение Batch Normalization
        output = self.dropout(output)     # Применение Dropout
        #***********************************************
        return output

# Создание экземпляра модели с параметрами из конфигурации
model = UBCModel(CONFIG['model_name'], CONFIG['num_classes'])
# Загрузка предобученных весов, если они есть
model.load_state_dict(torch.load(BEST_WEIGHT))
# Перемещение модели на устройство для вычислений (например, на GPU)
model.to(CONFIG['device']);


NameError: name 'nn' is not defined

In [ ]:
# Создание объекта тестового набора данных
test_dataset = UBCDataset(df_crop, transforms=data_transforms["valid"])

# Создание загрузчика данных для тестового набора
test_loader = DataLoader(
    test_dataset, 
    batch_size=CONFIG['valid_batch_size'],  # Размер пакета данных
    num_workers=6,                          # Количество рабочих процессов для загрузки данных
    shuffle=False,                          # Отсутствие перемешивания данных v4 false
    pin_memory=True                         # Предварительное выделение памяти, ускоряет передачу данных на GPU
)


In [ ]:
# Инициализация списка для сохранения предсказаний
preds = []

# Блокировка вычисления градиентов для экономии памяти и ускорения
with torch.no_grad():
    # Итерирование по тестовым данным с отображением прогресса
    bar = tqdm(enumerate(test_loader), total=len(test_loader))
    for step, data in bar:        
        # Перемещение изображений на устройство (например, GPU) и приведение к типу float
        images = data['image'].to(CONFIG["device"], dtype=torch.float)        
        # Получение размера пакета
        batch_size = images.size(0)
        # Вычисление предсказаний модели
        outputs = model(images)
        # Применение функции softmax для получения вероятностей классов
        outputs = model.softmax(outputs)
        # Добавление предсказаний в список preds
        preds.append(outputs.detach().cpu().numpy())

# Объединение результатов из разных пакетов в один массив
preds = np.vstack(preds)
print(preds.shape)


In [ ]:
# Добавление вероятностей каждого класса в DataFrame
for i in range(preds.shape[-1]):
    df_crop[f"cat{i}"] = preds[:, i]

# Инициализация словаря для окончательных предсказаний
dict_label = {}

# Группировка DataFrame по ID изображения и вычисление окончательного предсказания для каждого изображения
for image_id, gdf in df_crop.groupby("image_id"):
    # Выбор класса с максимальной вероятностью
    #dict_label[image_id] = np.argmax(gdf[[f"cat{i}" for i in range(preds.shape[-1])]].values.max(axis=0))
    # Альтернативный вариант: использование среднего значения вероятностей классов
    dict_label[image_id] = np.argmax(gdf[[f"cat{i}" for i in range(preds.shape[-1])]].values.mean(axis=0))

# Создание массива окончательных предсказаний, используя ID изображений из исходного DataFrame
preds = np.array([dict_label[image_id] for image_id in df["image_id"].values])


# submission

In [ ]:
# Использование обратного преобразования LabelEncoder для получения исходных текстовых меток
pred_labels = encoder.inverse_transform(preds)

# Присвоение этих меток в соответствующий столбец DataFrame
df_sub["label"] = pred_labels

# Сохранение DataFrame в файл CSV без индекса
df_sub.to_csv("submission.csv", index=False)

In [ ]:
df_sub